# Deep Learning: Ex.9 - Generative Adversarial Networks (GAN)

Submitted by: [... **namd and ID** ...]


In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten, Dropout, Input
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, LeakyReLU

print(tf.__version__)

---
### Loading the dataset

This dataset contains ~20K anime-face images (each image is 64x64 pixels):

In [ ]:
import gdown
from zipfile import ZipFile

url = "https://drive.google.com/uc?id=1ouH9kAGuZqTqknLvzDLiKRfCF0aqCROD"
gdown.download(url, 'data.zip', quiet=False)  # ~ 480MB

print('unzipping...')
with ZipFile("data.zip", "r") as zipobj:
    zipobj.extractall()  # unzip the file

print('done.')


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
dataGen = ImageDataGenerator(rescale=1./255)

dataset = dataGen.flow_from_directory('anime_data', target_size=(64,64), batch_size=25000, class_mode='sparse').next()[0]
print('dataset.shape =',dataset.shape)

# plot some samples:
plt.figure(figsize=(12,6))
for i in range(50):
		plt.subplot(5, 10, 1 + i)
		plt.axis('off')
		plt.imshow(dataset[i])

***

### 1. Training a GAN model 

- Use our model developed in the class presentation, and adjust its architecture to match the new dataset images shape: `(64,64,3)`.

- Use a latent dimension of 20-D as the noise input for the generator model.

- Generate 12 noise vectors (each one is of 20-D), and use these vectors to generate 12 images from the (un-trained) generator. Display the images in a single row of sublplots.



In [ ]:
###########################
###  your code here...  ###
###########################

- Train the GAN model for 50-100 epochs (it take ~1min/epoch).

- After each and every epoch, feed the **same** 12 noise vectors to the generator, and plot a generated images. This will help you follow the training progress. 


In [ ]:
###########################
###  your code here...  ###
###########################

***

### 2. Exploring the latent (input) space

- Generate and display 64 **new** images from the generator (use 8x8 subplots).

- Pick a single "good" image out of these images, and denote the seed-vector that generated it by `z`.

- Next, generate 25 new seed vectors, based on this specific `z`, in the following way:

 - We will add a small-variance noise vector `dz` (a.k.a pertubation, also of 20-D) to `z`, and feed the new vector (`z + dz`) to the generator, to generate a new image. 
  - Use a Normal distribution with $\sigma=0.5$ for the pertubation `dz` (you can experiment with different values for $\sigma$).

- Repeat this process to generate a total of 25 new images (each generated from the same seed `z` but with a different pertubation `dz`).

- Display the results (use 5x5 subplots)






In [ ]:
    ###########################
    ###  your code here...  ###
    ###########################

***
### 3. Morphing between two images


- Pick 2 out of the 64 examples you've generated in Q2. Denote the seed-vectors that generated these images by `z1` and `z2`.

- We will try to "morph" between these two images, by "walking" in the latent space from `z1` to `z2`.

 - Generate a set of 10 seed-vectors, using a linear interpolation (by varying $\alpha$ from 1 to 0):

$$ \vec{z}_{interp} \ \ =\ \  \alpha \cdot \vec{z_1} \ \ +\ \  (1-\alpha) \cdot \vec{z_2}$$

- Feed the interpolated seed vectors to the generator and display the sequence of images.



In [ ]:
###########################
###  your code here...  ###
###########################

- Extend the last process to morph between 3 different images `I1`, `I2`, `I3`:
 - Generate a morphing sequence from `I1` to `I2`, from `I2` to `I3`, and from `I3` back to `I1`.

In [ ]:
###########################
###  your code here...  ###
###########################

- Finally, if you wish, you can generate an animated GIF of your generated sequence:

In [ ]:
anim_file = 'animation.gif'

import imageio

with imageio.get_writer(anim_file, mode='I') as writer:
    # loop over the images you want to add to the animated GIF file
    # use: writer.append_data(your_image)
    # where your_image should be np.uint8 format, with values from 0-255.
  

***
## Good Luck!